In [13]:
import spotipy
import spotipy.util as util
import json
token =util.oauth2.SpotifyClientCredentials(client_id, client_secret)
cache_token = token.get_access_token()
spotify = spotipy.Spotify(cache_token)

tids = []
track_list = []
artist_list = ['Taylor Swift','Linkin Park','Charlie Puth']
for artist in artist_list:
    print(artist)
    results = spotify.search(q=artist,limit = 50)
    for i, t in enumerate(results['tracks']['items']):
        tids.append(t['uri'])

Taylor Swift
Linkin Park
Charlie Puth


In [14]:
tids

['spotify:track:29fRTIKsJhLUJoldM89GZS',
 'spotify:track:4MsUsFfWseB9M2plGJyuIw',
 'spotify:track:3Y7MHz19qCg1Pkxridtu1k',
 'spotify:track:6NFyWDv5CjfwuzoCkw47Xf',
 'spotify:track:5sOch9Cf4uxcMtwFzMtNqQ',
 'spotify:track:1vrd6UOGamcKNGnSHJQlSt',
 'spotify:track:5xTtaWoae3wi06K5WfVUUH',
 'spotify:track:1kHEuJRasudLhjvnbfc4yS',
 'spotify:track:1p80LdxRV74UKvL8gnD7ky',
 'spotify:track:1JbR9RDP3ogVNEWFgNXAjh',
 'spotify:track:3GCL1PydwsLodcpv0Ll1ch',
 'spotify:track:15DeqWWQB4dcEWzJg15VrN',
 'spotify:track:59HjlYCeBsxdI0fcm3zglw',
 'spotify:track:72jCZdH0Lhg93z6Z4hBjgj',
 'spotify:track:2x0WlnmfG39ZuDmstl9xfX',
 'spotify:track:2yLa0QULdQr0qAIvVwN6B5',
 'spotify:track:6FB3v4YcR57y4tXFcdxI1E',
 'spotify:track:1CiVz2WkmZRwhU3A6J2Gjz',
 'spotify:track:273dCMFseLcVsoSWx59IoE',
 'spotify:track:55n9yjI6qqXh5F2mYvUc2y',
 'spotify:track:7AEAGTc8cReDqcbPoY9gwo',
 'spotify:track:1P17dC1amhFzptugyAO7Il',
 'spotify:track:4svZDCRz4cJoneBpjpx8DJ',
 'spotify:track:4lIxdJw6W3Fg4vUIYCB0S5',
 'spotify:track:

In [15]:
results = spotify.search(q='Britney Spears',limit = 50)

In [16]:
results['tracks']['items']

[{'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/26dSoYclwsYLMAKD3tpOr4'},
     'href': 'https://api.spotify.com/v1/artists/26dSoYclwsYLMAKD3tpOr4',
     'id': '26dSoYclwsYLMAKD3tpOr4',
     'name': 'Britney Spears',
     'type': 'artist',
     'uri': 'spotify:artist:26dSoYclwsYLMAKD3tpOr4'}],
   'available_markets': ['AD',
    'AE',
    'AR',
    'AT',
    'AU',
    'BE',
    'BG',
    'BH',
    'BO',
    'BR',
    'CA',
    'CH',
    'CL',
    'CO',
    'CR',
    'CY',
    'CZ',
    'DE',
    'DK',
    'DO',
    'DZ',
    'EC',
    'EE',
    'EG',
    'ES',
    'FI',
    'FR',
    'GB',
    'GR',
    'GT',
    'HK',
    'HN',
    'HU',
    'ID',
    'IE',
    'IL',
    'IN',
    'IS',
    'IT',
    'JO',
    'JP',
    'KW',
    'LB',
    'LI',
    'LT',
    'LU',
    'LV',
    'MA',
    'MC',
    'MT',
    'MX',
    'MY',
    'NI',
    'NL',
    'NO',
    'NZ',
    'OM',
    'PA',
    'PE',
    'PH',
    'PL',
    'PS',
